# Merging Census and Health Data
Health data is saved as "health.csv" so change that if you have it labeled as something else.

To change variables extracted in census data, change codes under the "variables of interest section" and update the change name section.

To change variables extracted in census data, change "columns to keep" section.

In [2]:
import pandas as pd
import censusdata

## Census Data

In [4]:
# --- Variables of interest ---
variables = [
    'B00001_001E',  # Total population
    'B19013_001E',  # Median income
    'B17001_002E',  # Poverty
    'B15003_017E',  # HS grad
    'B15003_022E',  # Bachelor's degree
    'B23025_005E',  # Unemployed
    'B01002_001E'   # Median age
]

# --- Valid state FIPS codes (50 states + DC) ---
valid_states = [
    '01','02','04','05','06','08','09','10','11','12','13','15','16','17','18','19',
    '20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35',
    '36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53',
    '54','55','56'
]

# --- Function to parse censusgeo objects ---
def split_geo(cgeo):
    codes = dict(cgeo.geo)
    name_parts = cgeo.name.split(", ")
    tract_name = name_parts[0]
    county_name = name_parts[1]
    state_name = name_parts[2]
    return pd.Series([
        codes.get('state'), codes.get('county'), codes.get('tract'),
        state_name, county_name, tract_name
    ])

# --- Download and process data for all states ---
all_data = []

for state in valid_states:
    print(f"Downloading data for state FIPS: {state}")
    data = censusdata.download(
        'acs5', 2015,
        censusdata.censusgeo([('state', state), ('county', '*'), ('tract', '*')]),
        variables
    )
    
    # Extract geo info
    geo_cols = data.index.to_series().apply(split_geo)
    geo_cols.columns = [
        'State_FIPS', 'County_FIPS', 'Tract_FIPS',
        'State_Name', 'County_Name', 'Tract_Name'
    ]
    
    # Combine geo info with data
    data_clean = pd.concat([geo_cols, data], axis=1).reset_index(drop=True)
    all_data.append(data_clean)

# --- Combine all states ---
census_df = pd.concat(all_data, ignore_index=True)

# --- Create 11-digit tract FIPS ---
census_df["Tract_FIPS_full"] = (
    census_df["State_FIPS"].astype(str).str.zfill(2) +
    census_df["County_FIPS"].astype(str).str.zfill(3) +
    census_df["Tract_FIPS"].astype(str).str.zfill(6)
)

#rename variables
census_df.rename(columns={
    'B00001_001E': 'Census_Pop',
    'B19013_001E': 'Median_Income',
    'B17001_002E': 'Poverty_Count',
    'B15003_017E': 'HS_Grad_Count',
    'B15003_022E': 'Bachelors_Count',
    'B23025_005E': 'Unemployed_Count',
    'B01002_001E': 'Median_Age'
}, inplace=True)


print("ACS tract-level data ready with full 11-digit FIPS!")


ACS tract-level data ready with full 11-digit FIPS!


In [5]:
census_df.head()

,State_FIPS,County_FIPS,Tract_FIPS,State_Name,County_Name,Tract_Name,Census_Pop,Median_Income,Poverty_Count,HS_Grad_Count,Bachelors_Count,Unemployed_Count,Median_Age,Tract_FIPS_full
0,01,103,005109,Alabama,Morgan County,Census Tract 51.09,219.0,29644.0,1476,695,290,303,29.6,01103005109
1,01,103,005106,Alabama,Morgan County,Census Tract 51.06,225.0,35864.0,1186,899,924,293,43.3,01103005106
2,01,103,005107,Alabama,Morgan County,Census Tract 51.07,337.0,66739.0,137,632,805,126,43.5,01103005107
3,01,103,005108,Alabama,Morgan County,Census Tract 51.08,284.0,64632.0,566,589,535,166,45.8,01103005108
4,01,103,005701,Alabama,Morgan County,Census Tract 57.01,214.0,46306.0,571,295,205,52,38.4,01103005701


In [6]:
census_df.columns

Index(['State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Census_Pop', 'Median_Income', 'Poverty_Count',
       'HS_Grad_Count', 'Bachelors_Count', 'Unemployed_Count', 'Median_Age',
       'Tract_FIPS_full'],
      dtype='object')

## Health Data

In [8]:
health_df = pd.read_csv("health.csv")

In [9]:
health_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,5275,28.0,"(22.7, 33.2)",19.9,"(18.8, 20.9)",...,"(81.5, 86.0)",17.3,"(15.3, 19.2)",38.9,"(37.2, 40.4)",4.0,"( 3.5, 4.5)",18.8,"(12.6, 25.6)",POINT (-118.224698433 34.0470512474)
1,CA,Pasadena,656000,6037462001,0656000-06037462001,3974,23.3,"(19.2, 27.8)",16.7,"(16.0, 17.5)",...,"(84.1, 87.5)",15.3,"(13.9, 16.8)",40.2,"(38.8, 41.5)",3.2,"( 2.9, 3.5)",18.1,"(13.7, 22.8)",POINT (-118.143832177 34.1633689905)
2,CA,Fullerton,628000,6059011504,0628000-06059011504,5473,11.7,"( 9.6, 14.5)",8.3,"( 7.9, 8.7)",...,"(75.4, 80.9)",8.5,"( 7.7, 9.5)",33.4,"(31.6, 35.1)",1.2,"( 1.1, 1.3)",11.9,"( 8.8, 16.0)",POINT (-117.883112998 33.8820402343)
3,TX,Longview,4843888,48183000401,4843888-48183000401,3371,19.3,"(16.8, 21.9)",21.1,"(20.2, 21.9)",...,"(76.5, 81.9)",11.8,"(10.8, 12.8)",34.9,"(33.4, 36.3)",2.7,"( 2.5, 3.0)",15.1,"(10.8, 20.4)",POINT (-94.7523262965 32.5544549842)
4,WI,Madison,5548000,55025002200,5548000-55025002200,4254,10.5,"( 8.7, 12.6)",23.3,"(21.9, 24.6)",...,"(83.4, 88.0)",11.3,"( 9.8, 12.7)",30.8,"(29.3, 32.2)",2.9,"( 2.5, 3.3)",12.9,"( 7.6, 19.8)",POINT (-89.3623704161 43.1211036947)


In [10]:
#extract the 11-digit tract FIPS after the dash
health_df["Tract_FIPS_full"] = health_df["Place_TractID"].str.split("-").str[-1]

In [11]:
health_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,Tract_FIPS_full
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,5275,28.0,"(22.7, 33.2)",19.9,"(18.8, 20.9)",...,17.3,"(15.3, 19.2)",38.9,"(37.2, 40.4)",4.0,"( 3.5, 4.5)",18.8,"(12.6, 25.6)",POINT (-118.224698433 34.0470512474),06037206032
1,CA,Pasadena,656000,6037462001,0656000-06037462001,3974,23.3,"(19.2, 27.8)",16.7,"(16.0, 17.5)",...,15.3,"(13.9, 16.8)",40.2,"(38.8, 41.5)",3.2,"( 2.9, 3.5)",18.1,"(13.7, 22.8)",POINT (-118.143832177 34.1633689905),06037462001
2,CA,Fullerton,628000,6059011504,0628000-06059011504,5473,11.7,"( 9.6, 14.5)",8.3,"( 7.9, 8.7)",...,8.5,"( 7.7, 9.5)",33.4,"(31.6, 35.1)",1.2,"( 1.1, 1.3)",11.9,"( 8.8, 16.0)",POINT (-117.883112998 33.8820402343),06059011504
3,TX,Longview,4843888,48183000401,4843888-48183000401,3371,19.3,"(16.8, 21.9)",21.1,"(20.2, 21.9)",...,11.8,"(10.8, 12.8)",34.9,"(33.4, 36.3)",2.7,"( 2.5, 3.0)",15.1,"(10.8, 20.4)",POINT (-94.7523262965 32.5544549842),48183000401
4,WI,Madison,5548000,55025002200,5548000-55025002200,4254,10.5,"( 8.7, 12.6)",23.3,"(21.9, 24.6)",...,11.3,"( 9.8, 12.7)",30.8,"(29.3, 32.2)",2.9,"( 2.5, 3.3)",12.9,"( 7.6, 19.8)",POINT (-89.3623704161 43.1211036947),55025002200


In [12]:
health_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Population2010', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI',
       'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev',
       'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI',
       'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev',
       'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI',
       'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev',
       'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI',
       'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev',
       'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI',
       'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev',
       'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI',
       'DIABETES_CrudePrev', 'DIABETES_Crude95CI', 'HIGHCHOL_CrudePrev',
       'HIGHCHOL_Crude95CI', 'KIDNEY_CrudePrev', 'KIDNEY_Crude95CI',
       'LPA_CrudePrev', 'LPA_

In [13]:
#filter for columns we want
columns_to_keep = [
    "StateAbbr",
    "PlaceName",
    "PlaceFIPS",
    "TractFIPS",
    "Place_TractID",
    "Tract_FIPS_full",
    "Population2010",
    "OBESITY_CrudePrev",
    "DIABETES_CrudePrev",
    "HIGHCHOL_CrudePrev",
    "BPHIGH_CrudePrev",
    "STROKE_CrudePrev",
    "SLEEP_CrudePrev",
    "MHLTH_CrudePrev",
    "CASTHMA_CrudePrev",
    "CHD_CrudePrev",
    "CSMOKING_CrudePrev"
]

# Filter merged dataframe
health_filtered = health_df[columns_to_keep]

## Merge

In [15]:
merged_df = health_filtered.merge(census_df, on="Tract_FIPS_full", how="inner")

In [16]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Population2010', 'OBESITY_CrudePrev',
       'DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev', 'BPHIGH_CrudePrev',
       'STROKE_CrudePrev', 'SLEEP_CrudePrev', 'MHLTH_CrudePrev',
       'CASTHMA_CrudePrev', 'CHD_CrudePrev', 'CSMOKING_CrudePrev',
       'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Census_Pop', 'Median_Income', 'Poverty_Count',
       'HS_Grad_Count', 'Bachelors_Count', 'Unemployed_Count', 'Median_Age'],
      dtype='object')

In [17]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Population2010,OBESITY_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,...,State_Name,County_Name,Tract_Name,Census_Pop,Median_Income,Poverty_Count,HS_Grad_Count,Bachelors_Count,Unemployed_Count,Median_Age
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,California,Los Angeles County,Census Tract 2060.32,375.0,26094.0,1829,675,309,185,29.5
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,California,Los Angeles County,Census Tract 4620.01,264.0,40192.0,947,475,367,167,29.4
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,California,Orange County,Census Tract 115.04,435.0,31793.0,1740,225,873,379,22.8
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,Texas,Gregg County,Census Tract 4.01,255.0,52846.0,454,637,201,123,34.0
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,Wisconsin,Dane County,Census Tract 22,211.0,42621.0,491,523,594,339,42.1


In [18]:
merged_df.shape

(27209, 30)

## Cleaning

In [20]:
#rename Crude Columns
merged_df = merged_df.rename(columns={
    "Population2010" : "Health_Pop",
    "OBESITY_CrudePrev": "Obesity_Pct",
    "DIABETES_CrudePrev": "Diabetes_Pct",
    "HIGHCHOL_CrudePrev": "HighChol_Pct",
    "BPHIGH_CrudePrev": "HighBP_Pct",
    "STROKE_CrudePrev": "Stroke_Pct",
    "SLEEP_CrudePrev": "Sleep_Pct",
    "MHLTH_CrudePrev": "MentalHealth_Pct",
    "CASTHMA_CrudePrev": "Asthma_Pct",
    "CHD_CrudePrev": "HeartDisease_Pct",
    "CSMOKING_CrudePrev": "Smoking_Pct"
})

In [21]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Health_Pop,Obesity_Pct,Diabetes_Pct,HighChol_Pct,...,State_Name,County_Name,Tract_Name,Census_Pop,Median_Income,Poverty_Count,HS_Grad_Count,Bachelors_Count,Unemployed_Count,Median_Age
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,California,Los Angeles County,Census Tract 2060.32,375.0,26094.0,1829,675,309,185,29.5
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,California,Los Angeles County,Census Tract 4620.01,264.0,40192.0,947,475,367,167,29.4
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,California,Orange County,Census Tract 115.04,435.0,31793.0,1740,225,873,379,22.8
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,Texas,Gregg County,Census Tract 4.01,255.0,52846.0,454,637,201,123,34.0
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,Wisconsin,Dane County,Census Tract 22,211.0,42621.0,491,523,594,339,42.1


In [22]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Health_Pop', 'Obesity_Pct', 'Diabetes_Pct',
       'HighChol_Pct', 'HighBP_Pct', 'Stroke_Pct', 'Sleep_Pct',
       'MentalHealth_Pct', 'Asthma_Pct', 'HeartDisease_Pct', 'Smoking_Pct',
       'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Census_Pop', 'Median_Income', 'Poverty_Count',
       'HS_Grad_Count', 'Bachelors_Count', 'Unemployed_Count', 'Median_Age'],
      dtype='object')

In [23]:
merged_df.shape

(27209, 30)

In [24]:
#make new variables for raw count (health data)
#using the Health_Pop (from the health data)
for col in ["Obesity_Pct", "Diabetes_Pct", "HighChol_Pct", "HighBP_Pct",
            "Stroke_Pct", "Sleep_Pct", "MentalHealth_Pct",
            "Asthma_Pct", "HeartDisease_Pct", "Smoking_Pct"]:
    new_col = col.replace("_Pct", "_Count")
    merged_df[new_col] = round((merged_df[col] / 100) * merged_df["Health_Pop"])

In [25]:
#check the raw numbers
merged_df[["Health_Pop", "Obesity_Pct", "Obesity_Count"]].head()

,Health_Pop,Obesity_Pct,Obesity_Count
0,5275,29.5,1556.0
1,3974,32.0,1272.0
2,5473,18.7,1023.0
3,3371,33.9,1143.0
4,4254,29.5,1255.0


In [26]:
#make new variables for percentages (census data)
#using the Census_Pop (from census data)
for col in ["Poverty_Count", "HS_Grad_Count", "Bachelors_Count", "Unemployed_Count"]:
    new_col = col.replace("_Count", "_Pct")
    merged_df[new_col] = (merged_df[col] / merged_df["Census_Pop"]) * 100

In [27]:
#check the raw numbers
merged_df[["Census_Pop", "HS_Grad_Pct", "HS_Grad_Count"]].head()

,Census_Pop,HS_Grad_Pct,HS_Grad_Count
0,375.0,180.000000,675
1,264.0,179.924242,475
2,435.0,51.724138,225
3,255.0,249.803922,637
4,211.0,247.867299,523


In [28]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Health_Pop,Obesity_Pct,Diabetes_Pct,HighChol_Pct,...,Stroke_Count,Sleep_Count,MentalHealth_Count,Asthma_Count,HeartDisease_Count,Smoking_Count,Poverty_Pct,HS_Grad_Pct,Bachelors_Pct,Unemployed_Pct
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,211.0,2052.0,791.0,448.0,364.0,823.0,487.733333,180.000000,82.400000,49.333333
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,127.0,1598.0,624.0,374.0,191.0,707.0,358.712121,179.924242,139.015152,63.257576
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,66.0,1828.0,772.0,504.0,115.0,728.0,400.000000,51.724138,200.689655,87.126437
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,91.0,1176.0,455.0,314.0,182.0,691.0,178.039216,249.803922,78.823529,48.235294
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,123.0,1310.0,502.0,391.0,255.0,706.0,232.701422,247.867299,281.516588,160.663507


In [29]:
#check % of missing data
merged_df.isna().mean().sort_values(ascending=False).head(10)

Median_Income         0.001323
Unemployed_Pct        0.000809
Bachelors_Pct         0.000809
HS_Grad_Pct           0.000809
Poverty_Pct           0.000809
Census_Pop            0.000662
HeartDisease_Count    0.000000
Asthma_Count          0.000000
MentalHealth_Count    0.000000
Sleep_Count           0.000000
dtype: float64

In [30]:
#shape without NAs (less than 100 rows lost)
merged_df = merged_df.dropna()
merged_df.shape

(27151, 44)

In [31]:
#check % of missing data (should be 0s)
merged_df.isna().mean().sort_values(ascending=False).head(10)

StateAbbr           0.0
PlaceName           0.0
Median_Income       0.0
Poverty_Count       0.0
HS_Grad_Count       0.0
Bachelors_Count     0.0
Unemployed_Count    0.0
Median_Age          0.0
Obesity_Count       0.0
Diabetes_Count      0.0
dtype: float64

In [32]:
#reorder columns for better readability
# List the columns in the order you want
cols = [
    'StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
    'Tract_FIPS_full', 'Health_Pop',
    'Obesity_Pct', 'Obesity_Count',
    'Diabetes_Pct', 'Diabetes_Count',
    'HighChol_Pct', 'HighChol_Count',
    'HighBP_Pct', 'HighBP_Count',
    'Stroke_Pct', 'Stroke_Count',
    'Sleep_Pct', 'Sleep_Count',
    'MentalHealth_Pct', 'MentalHealth_Count',
    'Asthma_Pct', 'Asthma_Count',
    'HeartDisease_Pct', 'HeartDisease_Count',
    'Smoking_Pct', 'Smoking_Count',
    'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
    'Tract_Name', 'Census_Pop', 'Median_Income',
    'Poverty_Pct', 'Poverty_Count',
    'HS_Grad_Pct', 'HS_Grad_Count',
    'Bachelors_Pct', 'Bachelors_Count',
    'Unemployed_Pct', 'Unemployed_Count',
    'Median_Age'
]

# Reorder the DataFrame
merged_df = merged_df[cols]

In [33]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Health_Pop', 'Obesity_Pct', 'Obesity_Count',
       'Diabetes_Pct', 'Diabetes_Count', 'HighChol_Pct', 'HighChol_Count',
       'HighBP_Pct', 'HighBP_Count', 'Stroke_Pct', 'Stroke_Count', 'Sleep_Pct',
       'Sleep_Count', 'MentalHealth_Pct', 'MentalHealth_Count', 'Asthma_Pct',
       'Asthma_Count', 'HeartDisease_Pct', 'HeartDisease_Count', 'Smoking_Pct',
       'Smoking_Count', 'State_FIPS', 'County_FIPS', 'Tract_FIPS',
       'State_Name', 'County_Name', 'Tract_Name', 'Census_Pop',
       'Median_Income', 'Poverty_Pct', 'Poverty_Count', 'HS_Grad_Pct',
       'HS_Grad_Count', 'Bachelors_Pct', 'Bachelors_Count', 'Unemployed_Pct',
       'Unemployed_Count', 'Median_Age'],
      dtype='object')

## Save as CSV

In [35]:
#save as csv (won't have to run merging and cleaning again)
merged_df.to_csv("merged_health_census_clean.csv", index=False)